In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import json
from scipy import stats

In [2]:
filename = "rewiring results final.csv"

In [3]:
#create one aggregate file
import os
aggrfile = open("aggregate/{}".format(filename), "w+")
header = "graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist\n"
aggrfile.write(header)
aggrfile.close()
aggrfile = open("aggregate/{}".format(filename), "a+")
for filename in os.listdir("aggregate/"):
    if filename.startswith('averages triangles rewiring'):
        infile = open("aggregate/{}".format(filename), "r")
        lines = infile.readlines()
        for line in lines:
            l = line.strip().split(',')
            s = ','.join(l)
            s+='\n'
            aggrfile.write(s)
        infile.close()
aggrfile.close()

In [4]:
# results = pd.read_csv("aggregate/{}".format(filename))
# results.drop_duplicates(subset = ['graph', 'n', 'p', 'eps', 'gam', 'gr', 'pr', 'max_it'], inplace=True)
# results.to_csv("aggregate/{}".format(filename))
results = pd.read_csv(f"aggregate/{filename}")
results.head()

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist
0,ba,250,5.0,0.2,0.0,0.0,0.0,1000000,1.220333,0.045048,0.036204,0.120664
1,ba,250,5.0,0.2,0.2,0.0,0.0,1000000,1.248348,0.038644,0.037257,0.118065
2,ba,250,5.0,0.2,0.4,0.0,0.0,1000000,1.224998,0.049523,0.035013,0.115352
3,ba,250,5.0,0.2,0.6,0.0,0.0,1000000,1.260038,0.072474,0.039846,0.123714
4,ba,250,5.0,0.2,0.8,0.0,0.0,1000000,1.243255,0.049646,0.036577,0.116500


In [5]:
for attr in list(results.columns):
    print(attr)
    print(results[attr].unique())

graph
['ba' 'er']
n
[250]
p
[5.  0.1]
eps
[0.2 0.3 0.4]
gam
[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6]
gr
[0.]
pr
[0.  0.1 0.2 0.3 0.4 0.5]
max_it
[1000000]
avg_ncluster
[1.22033273 1.24834806 1.22499823 1.26003836 1.24325478 1.24732828
 1.2272389  1.22138881 1.20551834 1.08334169 1.10549896 1.08282673
 1.0996874  1.07878987 1.08223321 1.06984492 1.07683059 1.07612124
 1.02282967 1.02289885 1.02116054 1.02370936 1.01538074 1.01379731
 1.01705951 1.00968707 1.00969325 1.23511917 1.33203413 1.28208068
 1.02809281 1.05439201 1.03607253 1.06871972 1.02202424 1.01918618
 1.0024096  1.008032   1.0088449  1.00806078 1.00728365 1.
 1.0008032  1.03112738 1.03527851 1.03509744 1.03965579 1.04535976
 1.03894394 1.03866407 1.03875715 1.05513748 1.02208141 1.01290611
 1.01538711 1.05981121 1.03650799 1.03201204 1.0337028  1.03201204
 1.01453522 1.01542718 1.01372232 1.01291912 1.04548843 1.02364483
 1.01702332 1.02033092 1.00407993 1.04631975 1.05215411 1.06767112
 1.07551257 1.09593409 1.10395327 1.095

In [6]:
def nclusters_meanshift(data, bw):
    ops = {i: data[i] for i in range(len(data))}
    sorted_ops = sorted(ops.items(), key = lambda kv:(kv[1], kv[0]))
    A=np.array([el[1] for el in sorted_ops]).reshape(-1,1)
    clustering = MeanShift(bandwidth=bw).fit(A)
    lbls = clustering.labels_
    labels = np.arange(len(sorted_ops))
    for i in range(len(labels)):
        cl = lbls[i]
        labels[sorted_ops[i][0]]=cl
    cluster_participation_dict = {}
    for l in labels:
        if l not in cluster_participation_dict:
            cluster_participation_dict[l] = 1
        else:
            cluster_participation_dict[l] += 1
    #computing effective number of clusters using function explained in the paper
    C_num = 0
    C_den = 0
    for k in cluster_participation_dict:
        C_num += cluster_participation_dict[k]
        C_den += ((cluster_participation_dict[k])**2)
    C_num = (C_num**2)
    C = C_num/C_den
    return C

def nclusters(data, threshold):
    data = [float(el) for el in data]
    data = sorted(data)
    start = data[0]
    max_val = start + threshold
    c = (start, max_val)
    cluster = dict()
    for i in data:
        if i <= max_val:
            if c in cluster.keys():
                cluster[c] += 1
            else:
                cluster[c] = 1
        else:
            max_val = i + threshold
            c = (i, max_val)
            cluster[c] = 1
    #ora ho il dizionario con i cluster di una run
    C_num = len(data)**2
    C_den = 0
    for k in cluster.keys():
        C_den += cluster[k]*cluster[k]
    C = C_num / C_den
    return C

In [ ]:
# results = pd.read_csv("aggregate/{}".format(filename), index_col=[0])
# results.head()

In [7]:
new_values_avg = []
new_values_std = []
for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    gr = row['gr']
    graph = row['graph']
    if graph == "ba":
        p = int(row['p'])
    else:
        p = row['p']
    name = f'triangles rewiring {graph}{p} pr{pr} e{eps} g{gam} gr{gr} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc = nclusters_meanshift(finalops, bw=0.01)
        ncarray.append(nc)
    ncarray = np.array(ncarray)
    new_values_avg.append(np.average(ncarray))
    new_values_std.append(np.std(ncarray))

doing triangles rewiring ba5 pr0.0 e0.2 g0.0 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g0.2 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g0.4 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g0.6 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g0.8 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g1.0 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g1.2 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g1.4 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.2 g1.6 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.3 g0.0 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.3 g0.2 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.3 g0.4 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.3 g0.6 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.3 g0.8 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.3 g1.0 gr0.0 mi1000000
doing triangles rewiring ba5 pr0.0 e0.3 g1.2 gr0.0 mi1000000
doing triangles rewiring

In [8]:
results["0.01MS_avg_ncluster"] = new_values_avg
results["0.01MS_std_ncluster"] = new_values_std

In [15]:
new_values_avg = []
new_values_std = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    gr = row['gr']
    name = f'triangles rewiring {graph}{p} pr{pr} e{eps} g{gam} gr{gr} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    ncarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        nc = nclusters_meanshift(finalops, bw=0.00001)
        ncarray.append(nc)
    ncarray = np.array(ncarray)
    new_values_avg.append(np.average(ncarray))
    new_values_std.append(np.std(ncarray))

results["0.00001MS_avg_ncluster"] = new_values_avg
results["0.00001MS_std_ncluster"] = new_values_std

doing triangles rewiring er0.1 pr0.0 e0.2 g0.0 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.2 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.4 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.6 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g0.8 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.0 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.2 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.4 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.2 g1.6 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.0 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.2 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.4 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.6 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.3 g0.8 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.0 gr0.0 mi1000000
doing triangles rewiring er0.1 pr0.0 e0.3 g1.2 gr0.0 mi

In [ ]:
results.to_csv(f"aggregate/{filename}")

In [ ]:
results = pd.read_csv(f"aggregate/{filename}", index_col=[0])

In [ ]:
def entropy(opinions, n, nbins):
    bincounts, bin_edges = np.histogram(opinions, bins = np.linspace(0, 1, nbins))
    probabilities = bincounts/n
    entr = stats.entropy(probabilities)
    return entr

# def nodeperc_withininterval(opinions, center, width):
#     sortedops = sorted(opinions)
#     sortedopsarr = np.array(sortedops)
#     withinrange = np.where(np.logical_and(sortedopsarr>=(center-width), sortedopsarr<=(center+width)))
#     withinrange = withinrange[0]
#     perc = len(withinrange) / len(sortedops)
#     return perc

def average_opinions(opinions):
    a = np.array(opinions)
    return np.average(a)

def std_opinions(opinions):
    a = np.array(opinions)
    return np.std(a)

# def median_opinions(opinions):
#     a = np.array(opinions)
#     return np.median(a)

In [ ]:
results = pd.read_csv("aggregate/{}".format(filename), index_col=[0])
results.head()

In [ ]:
averageops = []
averagestdops = []
# averagemedianops = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    gr = row['gr']
    name = f'triangles rewiring {graph}{p} pr{pr} e{eps} g{gam} gr{gr} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    avgarray = []
    stdarray = []
    # medarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        avg = average_opinions(finalops)
        # med = median_opinions(finalops)
        std = std_opinions(finalops)
        avgarray.append(avg)
        # medarray.append(med)
        stdarray.append(std)
    avgarray = np.array(avgarray)
    # medarray = np.array(medarray)
    stdarray = np.array(stdarray)
    averageops.append(np.average(avgarray))
    averagestdops.append(np.average(stdarray))
    # averagemedianops.append(np.average(medarray))
results["avg_opinion"] = averageops
results["std_opinions"] = averagestdops

doing rewiring er0.1 pr0.0 e0.2 g0.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.8 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.8 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.4 g0.0 gr0.0 mi1

In [ ]:
values_avg = []
values_std = []

for index, row in results.iterrows():
    eps = row['eps']
    gam = row['gam']
    pr = row['pr']
    gr = row['gr']
    name = f'triangles rewiring {graph}{p} pr{pr} e{eps} g{gam} gr{gr} mi1000000'
    print(f'doing {name}')
    jsonfile = open(f'aggregate/final_opinions {name}.json')
    data = json.load(jsonfile)
    entrarray = []
    for nr in data.keys():
        finalops = list(data[nr])
        entr = entropy(finalops, 100, 101)
        entrarray.append(entr)
    entrarray = np.array(entrarray)
    values_avg.append(np.average(entrarray))
    values_std.append(np.std(entrarray))

results["10B_avg_entr"] = values_avg
results["10B_std_entr"] = values_std

# values_avg_2 = []
# values_std_2 = []

# for index, row in results.iterrows():
#     eps = row['eps']
#     gam = row['gam']
#     pr = row['pr']
#     gr = row['gr']
#     name = f'rewiring pr{pr} e{eps} g{gam} gr{gr} mi1000000'
#     print(f'doing {name}')
#     jsonfile = open(f'aggregate/final_opinions {name}.json')
#     data = json.load(jsonfile)
#     entrarray = []
#     for nr in data.keys():
#         finalops = list(data[nr])
#         entr = entropy(finalops, 100, 101)
#         entrarray.append(entr)
#     entrarray = np.array(entrarray)
#     values_avg_2.append(np.average(entrarray))
#     values_std_2.append(np.std(entrarray))

doing rewiring er0.1 pr0.0 e0.2 g0.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g0.8 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.2 g1.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g0.8 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.0 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.2 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.4 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.3 g1.6 gr0.0 mi1000000
doing rewiring er0.1 pr0.0 e0.4 g0.0 gr0.0 mi1

In [ ]:
results.head()

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter,0.01MS_avg_ncluster,0.01MS_std_ncluster,avg_opinion,std_opinions,10B_avg_entr,10B_std_entr
0,ba,250,5.0,0.2,0.0,0.0,0.0,1000000,2.104296,0.233641,0.116802,0.203618,103.22,45.185941,2.099349,0.229066,0.499160,0.233667,0.761646,0.106087
1,ba,250,5.0,0.2,0.2,0.0,0.0,1000000,2.401716,0.355721,0.121666,0.207808,131.54,62.631263,2.286651,0.289342,0.497057,0.236862,1.022523,0.169622
2,ba,250,5.0,0.2,0.4,0.0,0.0,1000000,2.559789,0.509262,0.120890,0.206556,118.27,45.827709,2.388097,0.427533,0.504159,0.235085,1.137151,0.204202
3,ba,250,5.0,0.2,0.6,0.0,0.0,1000000,2.766177,0.569425,0.124099,0.209612,123.35,50.127590,2.555332,0.462896,0.499872,0.236304,1.242036,0.213477
4,ba,250,5.0,0.2,0.8,0.0,0.0,1000000,2.961323,0.611544,0.124814,0.207793,157.35,98.495488,2.786033,0.510264,0.502227,0.235374,1.509094,0.213801


In [3]:
# results.to_csv("aggregate/{}".format(filename))
results = pd.read_csv("aggregate/{}".format(filename), index_col=[0])

In [4]:
results.head()

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,avg_pwdist,std_pwdist,avg_niter,std_niter,0.01MS_avg_ncluster,0.01MS_std_ncluster,avg_opinion,std_opinions,10B_avg_entr,10B_std_entr
0,ba,250,5.0,0.2,0.0,0.0,0.0,1000000,2.104296,0.233641,0.116802,0.203618,103.22,45.185941,2.099349,0.229066,0.499160,0.233667,0.761646,0.106087
1,ba,250,5.0,0.2,0.2,0.0,0.0,1000000,2.401716,0.355721,0.121666,0.207808,131.54,62.631263,2.286651,0.289342,0.497057,0.236862,1.022523,0.169622
2,ba,250,5.0,0.2,0.4,0.0,0.0,1000000,2.559789,0.509262,0.120890,0.206556,118.27,45.827709,2.388097,0.427533,0.504159,0.235085,1.137151,0.204202
3,ba,250,5.0,0.2,0.6,0.0,0.0,1000000,2.766177,0.569425,0.124099,0.209612,123.35,50.127590,2.555332,0.462896,0.499872,0.236304,1.242036,0.213477
4,ba,250,5.0,0.2,0.8,0.0,0.0,1000000,2.961323,0.611544,0.124814,0.207793,157.35,98.495488,2.786033,0.510264,0.502227,0.235374,1.509094,0.213801


In [8]:
#normalize entropy
import pandas as pd
from sklearn import preprocessing

x = results['10B_avg_entr'].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.reshape(-1, 1))
results['10B_avg_entr_norm'] = pd.DataFrame(x_scaled)
results.head()

,graph,n,p,eps,gam,gr,pr,max_it,avg_ncluster,std_ncluster,...,std_pwdist,avg_niter,std_niter,0.01MS_avg_ncluster,0.01MS_std_ncluster,avg_opinion,std_opinions,10B_avg_entr,10B_std_entr,10B_avg_entr_norm
0,ba,250,5.0,0.2,0.0,0.0,0.0,1000000,2.104296,0.233641,...,0.203618,103.22,45.185941,2.099349,0.229066,0.499160,0.233667,0.761646,0.106087,0.165136
1,ba,250,5.0,0.2,0.2,0.0,0.0,1000000,2.401716,0.355721,...,0.207808,131.54,62.631263,2.286651,0.289342,0.497057,0.236862,1.022523,0.169622,0.227483
2,ba,250,5.0,0.2,0.4,0.0,0.0,1000000,2.559789,0.509262,...,0.206556,118.27,45.827709,2.388097,0.427533,0.504159,0.235085,1.137151,0.204202,0.254877
3,ba,250,5.0,0.2,0.6,0.0,0.0,1000000,2.766177,0.569425,...,0.209612,123.35,50.127590,2.555332,0.462896,0.499872,0.236304,1.242036,0.213477,0.279944
4,ba,250,5.0,0.2,0.8,0.0,0.0,1000000,2.961323,0.611544,...,0.207793,157.35,98.495488,2.786033,0.510264,0.502227,0.235374,1.509094,0.213801,0.343768


In [11]:
results["10B_avg_entr_norm"].min()

0.0

In [12]:
results["10B_avg_entr_norm"].max()

1.0

In [13]:
results.to_csv("aggregate/{}".format(filename))